## Prompt Chaining Workflow

Prompt chaining is a technique where complex tasks are broken into sequential steps, with each step's output feeding into the next. This approach enables:

- Better context management for complex workflows
- Modular and reusable components
- Easier debugging and validation at each stage
- Step-by-step reasoning similar to human problem-solving

We use StateGraph to define nodes (individual prompts/tasks) and edges (information flow) to create these chains.

### Environment Setup

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

### Initialize LLM

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7)

/Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define State Schema

State tracks data flow through the chain. Each node can read from and write to state.

In [3]:
from typing_extensions import TypedDict

class ResearchState(TypedDict):
    topic: str
    outline: str
    draft: str
    final_report: str
    quality_score: int

### Define Chain Nodes

Each node represents a specific step in the prompt chain. Nodes are pure functions that take state and return updates.

In [4]:
def create_outline(state: ResearchState) -> dict:
    """Generate structured outline for the topic"""
    prompt = f"Create a detailed 3-section outline for a research report on: {state['topic']}. Include key points for each section."
    response = llm.invoke(prompt)
    return {"outline": response.content}

In [5]:
def write_draft(state: ResearchState) -> dict:
    """Expand outline into full draft"""
    prompt = f"""Based on this outline:
{state['outline']}

Write a comprehensive draft report on {state['topic']}. Expand each section with detailed analysis."""
    response = llm.invoke(prompt)
    return {"draft": response.content}

In [6]:
def evaluate_quality(state: ResearchState) -> dict:
    """Assess draft quality on 1-10 scale"""
    prompt = f"""Rate this draft report from 1-10 based on clarity, depth, and structure:
{state['draft'][:500]}...

Respond with only a number between 1-10."""
    response = llm.invoke(prompt)
    try:
        score = int(response.content.strip())
    except:
        score = 5
    return {"quality_score": score}

In [7]:
def refine_report(state: ResearchState) -> dict:
    """Polish and enhance the draft"""
    prompt = f"""Improve this draft report by:
- Enhancing clarity and flow
- Adding transitional phrases
- Strengthening conclusions

Draft:
{state['draft']}"""
    response = llm.invoke(prompt)
    return {"final_report": response.content}

### Conditional Routing Logic

Routes flow based on quality score - demonstrates conditional chains.

In [8]:
def route_by_quality(state: ResearchState) -> str:
    """Route to refinement if quality is good, otherwise regenerate"""
    if state["quality_score"] >= 7:
        return "refine"
    return "rewrite"

### Build the Workflow Graph

Construct the chain by defining nodes and their connections.

In [9]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(ResearchState)

# Add nodes for each step
workflow.add_node("outline", create_outline)
workflow.add_node("draft", write_draft)
workflow.add_node("evaluate", evaluate_quality)
workflow.add_node("refine", refine_report)
workflow.add_node("rewrite", write_draft)

### Define Edge Connections

Edges control the flow between nodes in the chain.

In [10]:
# Linear chain: START -> outline -> draft -> evaluate
workflow.add_edge(START, "outline")
workflow.add_edge("outline", "draft")
workflow.add_edge("draft", "evaluate")

# Conditional routing based on quality
workflow.add_conditional_edges(
    "evaluate",
    route_by_quality,
    {"refine": "refine", "rewrite": "rewrite"}
)

# Both paths converge to END
workflow.add_edge("rewrite", END)
workflow.add_edge("refine", END)

### Compile and Visualize

In [11]:
from IPython.display import Image, display

app = workflow.compile()

# Visualize the workflow
try:
    graph_image = app.get_graph().draw_mermaid_png()
    display(Image(graph_image))
except Exception as e:
    print(f"Graph visualization unavailable: {e}")

Graph visualization unavailable: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


### Execute the Chain

Run the complete workflow with an input topic.

In [12]:
initial_state = {
    "topic": "Impact of RAG systems on enterprise knowledge management"
}

result = app.invoke(initial_state)

### Inspect Results

View outputs from each stage of the chain.

In [13]:
print("=" * 60)
print("OUTLINE")
print("=" * 60)
print(result["outline"])
print("\n" + "=" * 60)
print(f"QUALITY SCORE: {result['quality_score']}/10")
print("=" * 60)

OUTLINE
Here is a detailed 3-section outline for a research report on the impact of RAG (Red, Amber, Green) systems on enterprise knowledge management:

**Section 1: Introduction and Background**
- Define RAG systems and their purpose in enterprise knowledge management
- Provide an overview of the current state of enterprise knowledge management and its challenges
- Discuss the significance of RAG systems in addressing these challenges
- Introduce the research question: How do RAG systems impact enterprise knowledge management?
- Outline the objectives of the research report, including:
  * To examine the current use of RAG systems in enterprise knowledge management
  * To investigate the benefits and limitations of RAG systems
  * To identify best practices for implementing RAG systems in enterprise knowledge management
- Provide an overview of the methodology used in the research, including data collection and analysis methods

**Section 2: Literature Review and Analysis**
- Review e

In [14]:
# View final output
final_key = "final_report" if "final_report" in result else "draft"
print("=" * 60)
print("FINAL REPORT")
print("=" * 60)
print(result[final_key][:1000] + "...")

FINAL REPORT
**Section 1: Introduction and Background**

In today's fast-paced business environment, effective management of knowledge within an organization is crucial for its success and competitiveness. Enterprise knowledge management (EKM) refers to the processes and systems used to capture, organize, store, retrieve, and share knowledge and information within an organization. However, EKM faces numerous challenges, including the complexity of knowledge assets, the difficulty of capturing and sharing tacit knowledge, and the need for effective decision-making and risk management. To address these challenges, organizations have been seeking innovative approaches to improve their EKM practices.

One such approach that has gained popularity in recent years is the use of Red, Amber, Green (RAG) systems. RAG systems provide a simple yet effective way to categorize and prioritize knowledge assets, risks, and decisions based on their level of complexity, urgency, and impact. The purpose o

### Sequential Chaining
Each node processes previous outputs: outline → draft → evaluation → refinement

### State Management
TypedDict maintains context across the entire chain

### Conditional Logic
Quality-based routing shows dynamic chain behavior

### Modularity
Each node is independently testable and reusable

### Production Considerations
- Add error handling for each node
- Implement retry logic for failed steps
- Add logging for debugging
- Consider streaming for long-running chains
- Use checkpointing for complex workflows